# Notebook for development and testing

In [1]:
from DataLoader import *

data = ship_dataset("/home/franko/Desktop/DSelimovcProject/Dataset/", 'train')
input, output, name = data[0]
print(input[0].shape)

torch.Size([3, 1501])


In [14]:
import torch
from torch import nn
from torch.utils.data import DataLoader
import torch.nn.functional as F
import math

In [3]:
def init_train_dataloader():
    """
        Init of the train data loader. NOT TESTED FOR MULTIPLE GPU
        Creating wrapper arround data class. 
    """
    _train_ds = ship_dataset("/home/franko/Desktop/DSelimovcProject/Dataset/", 'train')
    batch_size = 32
    if False and False:
        batch_size *= torch.cuda.device_count()

    _train_dl = DataLoader(
        _train_ds,
        batch_size=batch_size,
        num_workers=1,
        pin_memory=False,
    )  
    return _train_dl

In [15]:
class CNN_REG(nn.Module):
    def __init__(self, max_pooling_ratio = 2):
        super(CNN_REG, self).__init__()
        self.dense_number = math.floor((math.floor((1501-15+1) / max_pooling_ratio) - 9 + 1) / max_pooling_ratio)

        
        self.first_conv = torch.nn.Conv2d(1, 48, kernel_size = (3, 15))
        self.second_conv = torch.nn.Conv2d(48, 48, kernel_size = (1,9))
        self.dense = torch.nn.Linear(self.dense_number * 48, 30)
        self.dropout = nn.Dropout(p=0.25)
        self.output_layer = torch.nn.Linear(30, 3)
        
        self.max_pooling_ratio = max_pooling_ratio
        
    def forward(self, x):
        _out = self.first_conv(x)
        _out = torch.tanh(_out)
        _out = F.max_pool2d(_out, kernel_size = (1, self.max_pooling_ratio))
        _out = self.second_conv(_out)
        _out = torch.tanh(_out)
        _out = F.max_pool2d(_out, kernel_size = (1, self.max_pooling_ratio))
        
        
        _out = _out.view(-1, self.dense_number * 48)
        
        _out = self.dense(_out)
        _out = torch.tanh(_out)
        _out = self.dropout(_out)
        _out = self.output_layer(_out)
        _out = torch.tanh(_out)
        
        return _out

In [16]:
dl = init_train_dataloader()

for batch in dl:
    _input, _output, _ = batch
    break
model = CNN_REG(max_pooling_ratio = 2)
model = model.double()
print(_input.shape)
pred = model(_input)
print(pred.shape)


367
torch.Size([32, 1, 3, 1501])
torch.Size([32, 3])
